# 01 - clientes novos por regiao por periodo r


In [1]:
''' mes / centro oeste / ...... / sul / total
total / ..... /  '''

' mes / centro oeste / ...... / sul / total\ntotal / ..... /  '

In [2]:
import os,sys
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)
import pandas as pd
from Scripts.Reads import ReadConfigs
from Scripts.Padronizes import PadronizeProdutos
from pandas import to_datetime as pdDatetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas import to_numeric as pdNumeric
from collections import defaultdict
import xlsxwriter


In [3]:

# -- 1 - definindo dfs e colunas
base_dir = os.getcwd()
projeto_dir = os.path.abspath(os.path.join(base_dir, '..', '..'))
pasta_dados = os.path.abspath(os.path.join(projeto_dir,'Data', 'BRUTOS-GERAIS', 'CLIENTES'))
df_clientes_masquerade = pd.read_excel(r'C:\Users\MSI\OneDrive\Desktop\faturamento masquerade\Relatorios_faturamento_Masquerade\Data\BRUTOS-GERAIS\CLIENTES\sh-clientes_final.xlsx')
df_clientes = df_clientes_masquerade[['Cd Uf','Nm Cidade','Dt Cadastro']]
df_estados_regiões = pd.read_csv(r'C:\Users\MSI\OneDrive\Desktop\faturamento masquerade\Relatorios_faturamento_Masquerade\Data\BRUTOS-GERAIS\Regioes-estado.csv')
df_novos = ['regiao', 'Nm cidade', 'Quant']

# -- 2 - definindo a região 
df_clientes = df_clientes.merge(df_estados_regiões[['Cd Uf', 'Região']],
                                on='Cd Uf', how='left' )

# -- 3 definindo periodo 

ano, mes = ReadConfigs()

# -- 4 - filtrando pelo peridodo
primeiro_dia = datetime(ano, mes, 1)
ultimo_dia = (primeiro_dia + relativedelta(months=1)) - relativedelta(days=1)   
df_novos_raw = df_clientes[
        (df_clientes['Dt Cadastro'] >= primeiro_dia) & 
        (df_clientes['Dt Cadastro'] <= ultimo_dia)]
df_novos_raw['Ano'] = ano
df_novos_raw['Mês'] = mes
df_novos_raw['Periodo'] = df_novos_raw['Ano'].astype(str) + '-' + df_novos_raw['Mês'].astype(str).str.zfill(2)
df_novos_raw

C:\Users\MSI\AppData\Local\Temp\ipykernel_10800\748999245.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novos_raw['Ano'] = ano
C:\Users\MSI\AppData\Local\Temp\ipykernel_10800\748999245.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novos_raw['Mês'] = mes
C:\Users\MSI\AppData\Local\Temp\ipykernel_10800\748999245.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,Cd Uf,Nm Cidade,Dt Cadastro,Região,Ano,Mês,Periodo
160,MS,AQUIDAUANA,2025-06-26,Centro-Oeste,2025,6,2025-06
368,RJ,CABO FRIO,2025-06-25,Sudeste,2025,6,2025-06
454,MT,NOVA MUTUM,2025-06-26,Centro-Oeste,2025,6,2025-06
605,PA,BELEM,2025-06-30,Norte,2025,6,2025-06
736,MS,RIO BRILHANTE,2025-06-23,Centro-Oeste,2025,6,2025-06
1272,MS,NOVA ALVORADA DO SUL,2025-06-26,Centro-Oeste,2025,6,2025-06
1329,DF,BRASILIA,2025-06-02,Centro-Oeste,2025,6,2025-06
1401,RS,SANTO CRISTO,2025-06-25,Sul,2025,6,2025-06
1441,GO,GOIANIA,2025-06-20,Centro-Oeste,2025,6,2025-06
1515,MG,LAVRAS,2025-06-10,Sudeste,2025,6,2025-06


In [4]:

def gerar_relatorio_clientes_novos(df_novos):
    # Diretórios
    projeto_base = r'C:\Users\MSI\OneDrive\Desktop\faturamento masquerade\Relatorios_faturamento_Masquerade'
    base_path = os.path.join(projeto_base, 'resource', 'Clientes-novos', str(ano))
    pasta_atual = os.path.join(base_path, f"{mes:02d}")
    pasta_atual = os.path.join(base_path, f"{mes:02d}")
    os.makedirs(pasta_atual, exist_ok=True)
    arquivo_atual = os.path.join(pasta_atual, 'relatorio_clientes_novos.xlsx')

    colunas = ['Mês', 'Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste', 'Sul', 'Total', 'ultimo']

    # Função para contar clientes por região
    def calcular_linha(df, mes, ano):
        regiao_contagem = df['Região'].value_counts().to_dict()
        linha = {
            'Mês': f'{ano}-{mes:02d}',
            'Centro-Oeste': regiao_contagem.get('Centro-Oeste', 0),
            'Nordeste': regiao_contagem.get('Nordeste', 0),
            'Norte': regiao_contagem.get('Norte', 0),
            'Sudeste': regiao_contagem.get('Sudeste', 0),
            'Sul': regiao_contagem.get('Sul', 0),
            'Total': df.shape[0],
            'ultimo': True
        }
        return linha

    nova_linha = calcular_linha(df_novos, mes, ano)

    # Se não for janeiro e existir mês anterior, tenta carregar relatório anterior
    if mes > 1:
        pasta_ant = os.path.join(base_path, f"{mes - 1:02d}")
        arquivo_ant = os.path.join(pasta_ant, 'relatorio_clientes_novos.xlsx')

        if os.path.exists(arquivo_ant):
            df_antigo = pd.read_excel(arquivo_ant)
            if 'ultimo' in df_antigo.columns:
                idx_ultimo = df_antigo[df_antigo['ultimo'] == True].index
                if not idx_ultimo.empty:
                    df_antigo.at[idx_ultimo[0], 'ultimo'] = False
            df_final = pd.concat([df_antigo, pd.DataFrame([nova_linha])], ignore_index=True)
        else:
            df_final = pd.DataFrame([nova_linha], columns=colunas)
    else:
        df_final = pd.DataFrame([nova_linha], columns=colunas)

    # Salvando com xlsxwriter
    with pd.ExcelWriter(arquivo_atual, engine='xlsxwriter') as writer:
        df_final.to_excel(writer, sheet_name=f'RELATORIO CLIENTES NOVOS {ano}', index=False)

    print(f"[✔] Relatório salvo: {arquivo_atual}")

gerar_relatorio_clientes_novos(df_novos_raw)

[✔] Relatório salvo: C:\Users\MSI\OneDrive\Desktop\faturamento masquerade\Relatorios_faturamento_Masquerade\resource\Clientes-novos\2025\06\relatorio_clientes_novos.xlsx
